In [1]:
# Importing Packages
import pandas as pd
import numpy as np
import re
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
import statsmodels.api as sm
warnings.filterwarnings("ignore") 
import random
random.seed(10)

In [2]:
# Importing Datasets
df_bicycle = pd.read_csv('../Data/NYC_Bicycle_Counts_2016_Corrected.csv')

In [3]:
def preprocessing_df_1(X,Y):
    # Ordinal Encoding
    def ordinal_encoding(df,col,mapping):
        ordinal_encoder=ce.OrdinalEncoder(cols=col,return_df=True,mapping=[{'col':col,'mapping':mapping}])
        df_final = ordinal_encoder.fit_transform(df)
        return df_final
    # Preprocessing Data
    X = df_bicycle[X]
    Y = df_bicycle[Y]
    Y_Name = Y.columns[0]
    X = ordinal_encoding(X,"Day",{"Monday":1,"Tuesday":2,"Wednesday":3,"Thursday":4,"Friday":5,"Saturday":6,"Sunday":7})
    for i,j in enumerate(X["Precipitation"]):
        if j == "0.47 (S)":
            X["Precipitation"].iloc[i] = 0.47
        if j == "T":
            X["Precipitation"].iloc[i] = 0

    X = X.replace(',','', regex=True)
    Y = Y.replace(',','', regex=True)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    Y = scaler.fit_transform(Y)
    return X,Y,Y_Name

In [4]:
def Question1(X,Y):
    X,Y,Y_Name = preprocessing_df_1(X,Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=101)
    # Decision Tree Regressor
    model = DecisionTreeRegressor()
    model.fit(X_train,y_train)
    Y_pred = model.predict(X_test)
    MSE = round(np.sqrt(mean_squared_error(Y_pred,y_test)),3)
    print('RMSE with ' + Y_Name + ' as target variable is ' + str(MSE))

In [5]:
Question1(['Day','Precipitation','High Temp (°F)','Low Temp (°F)','Brooklyn Bridge','Manhattan Bridge','Williamsburg Bridge'],['Queensboro Bridge'])
Question1(['Day','Precipitation','High Temp (°F)','Low Temp (°F)','Brooklyn Bridge','Manhattan Bridge','Queensboro Bridge'],['Williamsburg Bridge'])
Question1(['Day','Precipitation','High Temp (°F)','Low Temp (°F)','Brooklyn Bridge','Queensboro Bridge','Williamsburg Bridge'],['Manhattan Bridge'])
Question1(['Day','Precipitation','High Temp (°F)','Low Temp (°F)','Queensboro Bridge','Manhattan Bridge','Williamsburg Bridge'],['Brooklyn Bridge'])

RMSE with Queensboro Bridge as target variable is 0.368
RMSE with Williamsburg Bridge as target variable is 0.295
RMSE with Manhattan Bridge as target variable is 0.819
RMSE with Brooklyn Bridge as target variable is 0.644


#### Here, we can that Brooklyn Bridge, Manhattan Bridge, Queensboro Bridge estimate the overall traffic with lower RMSE value .
####  Therefore, I choose install sensor on Brooklyn Bridge, Manhattan Bridge, and Queensboro Bridge

In [6]:
def preprocessing_df_2(X,Y):
    # Ordinal Encoding
    def ordinal_encoding(df,col,mapping):
        ordinal_encoder=ce.OrdinalEncoder(cols=col,return_df=True,mapping=[{'col':col,'mapping':mapping}])
        df_final = ordinal_encoder.fit_transform(df)
        return df_final
    # Preprocessing Data
    X = df_bicycle[X]
    Y = df_bicycle[Y]
    Y_Name = Y.columns[0]
    for i,j in enumerate(X["Precipitation"]):
        if j == "0.47 (S)":
            X["Precipitation"].iloc[i] = 0.47
        if j == "T":
            X["Precipitation"].iloc[i] = 0

    X = X.replace(',','', regex=True)
    Y = Y.replace(',','', regex=True)
    return X,Y,Y_Name

In [7]:
def Question2(X,Y):
    X,Y,Y_Name = preprocessing_df_2(X,Y)
    regression = LinearRegression()

    # Regression on the original data
    regression.fit(X[['High Temp (°F)','Low Temp (°F)','Precipitation']], Y)
    intercept =regression.intercept_
    coef = regression.coef_

    print(coef[0][0])
    X = sm.add_constant(X[['High Temp (°F)','Low Temp (°F)','Precipitation']])
    model = sm.OLS(Y.astype(float),X.astype(float)).fit()
    print(model.summary())
    print("Regression Equation :",np.round(coef[0][0],2),"* High Temp (°F)",np.round(coef[0][1],2),"* Low Temp (°F)",np.round(coef[0][2],2),"* Precipitation"," + ", np.round(intercept[0],2))

In [8]:
Question2(['Precipitation','High Temp (°F)','Low Temp (°F)'],["Total"])

390.9183083363229
                            OLS Regression Results                            
Dep. Variable:                  Total   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     69.85
Date:                Tue, 30 Nov 2021   Prob (F-statistic):           2.28e-31
Time:                        16:10:52   Log-Likelihood:                -2079.9
No. Observations:                 214   AIC:                             4168.
Df Residuals:                     210   BIC:                             4181.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            178.2009 

#### Here, we can see that R Squared Value = 0.499 for our model. From this we can state that 50 % of dependent varaible value can be explained by the independent variables. So we conclude that they can use weather data to predict bicyclists in a day

In [9]:
def preprocessing_df_3(X,Y):
    # Ordinal Encoding
    def ordinal_encoding(df,col,mapping):
        ordinal_encoder=ce.OrdinalEncoder(cols=col,return_df=True,mapping=[{'col':col,'mapping':mapping}])
        df_final = ordinal_encoder.fit_transform(df)
        return df_final
    # Preprocessing Data
    X = df_bicycle[X]
    Y = df_bicycle[Y]
    Y_Name = Y.columns[0]
    for i,j in enumerate(Y["Precipitation"]):
        if j == "0.47 (S)":
            Y["Precipitation"].iloc[i] = 0.47
        if j == "T":
            Y["Precipitation"].iloc[i] = 0

    Y["Precipitation"] = Y["Precipitation"].astype(float)
    for j,i in enumerate(Y["Precipitation"]):
        if i >= 0.5:
            Y["Precipitation"].iloc[j] = 1
        else:
            Y["Precipitation"].iloc[j] = 0
        
    X = X.replace(',','', regex=True)
    Y = Y.replace(',','', regex=True)
    return X,Y,Y_Name

In [10]:
def Question3(X,Y):
    X,Y,Y_Name = preprocessing_df_3(X,Y)
    X_train, X_test, y_train, y_test = train_test_split(X,Y,train_size=0.75,random_state= 5)
    log_model = LogisticRegression()
    log_model.fit(X_train,y_train)
    y_test = np.array(y_test)
    y_pred = log_model.predict(X_test)
    y_pred = np.array(y_pred)
    score = log_model.score(X_test,y_test)
    print("Coefficients : ",log_model.coef_[0][0])
    print("Intercepts : ",log_model.intercept_[0])
    print('Accuracy Score : ' +str(score))

In [11]:
Question3(["Total"],["Precipitation"])

Coefficients :  -0.0001407945211298256
Intercepts :  -3.006283242932824e-10
Accuracy Score : 0.9444444444444444


#### Here, we can see that accuracy is around 95% for our model. So we can use this data to predict whether it is raining based on the number of bicyclists on the bridges